In [1]:
# Live project
#files_bucket = "vf-gr-ca-live-proda"
#output_bucket_new = "vfgr-dh-ca-modeloutputs"

# NonLive project
files_bucket = "vf-gr-ca-nonlive-devde"
output_bucket_new = "vf-gr-ca-nonlive-devds/modeloutputs"
dhdwh_bucket = "vfgr-dh-dwh-rawprepared"

from pyspark.sql.functions import asc, desc

In [2]:
import re
import math
import warnings
import subprocess
from rtree import Rtree
warnings.filterwarnings("ignore")
import datetime
import operator
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from math import isnan

In [3]:
pd.set_option('display.max_columns', 550)
pd.set_option('display.max_rows', 500)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
from calendar import monthrange
import subprocess
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import Window
from pyspark.sql.functions import udf, split, col,count, sum, coalesce, lit, avg
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.functions import format_number,dayofmonth,hour,dayofyear,month,year,weekofyear,date_format
from pyspark.sql import functions as f
from pyspark.sql.functions import mean, min, max, stddev, countDistinct, sum, col, desc
#from pyspark.sql import functions as F

import pyspark.sql.functions as fn
import pyspark.sql.types as types
import pyspark.ml.feature as fe
from itertools import chain

import time
import datetime

import operator as op 
import functools as ft

In [5]:
# Get spark
config = SparkConf()

spark = SparkSession.builder.config(conf=config).getOrCreate()
spark

In [6]:
start_time = time.time()
print(start_time)

1687253384.4373477


In [7]:
currentDay = datetime.date.today().day
currentMonth = datetime.date.today().month - 3
currentYear = datetime.date.today().year

In [8]:
print(currentDay,currentMonth,currentYear)

20 3 2023


In [9]:
project_date = datetime.datetime(currentYear, currentMonth, 1)
print(project_date)

2023-03-01 00:00:00


In [10]:
import datetime
from dateutil.relativedelta import relativedelta
year_val = []
month_val = []
for x in (1,2,3,4,5):
        month = (project_date+relativedelta(months=-x)).date().month
        year = (project_date+relativedelta(months=-x)).date().year
        month_val.append(month)
        year_val.append(year)
        print(month_val, year_val)

[2] [2023]
[2, 1] [2023, 2023]
[2, 1, 12] [2023, 2023, 2022]
[2, 1, 12, 11] [2023, 2023, 2022, 2022]
[2, 1, 12, 11, 10] [2023, 2023, 2022, 2022, 2022]


In [11]:
def read_in_data_month(in_file, year=None , month=None):
    df = spark.read.parquet(in_file.format(year, month))
    return df

In [12]:
def read_in_data_day(in_file, year=None , month=None, day=None):
    df = spark.read.parquet(in_file.format(year, month, day))
    return df

In [13]:
in_file_1 = 'gs://' + dhdwh_bucket + '/vid_ope_hubs_userinfo/1.0/parquet/year={}/month={}/day={}'
in_file_2 = 'gs://' + dhdwh_bucket + '/vid_ope_v_dslam_users/1.0/parquet/year={}/month={}/day={}'

in_file_3 = 'gs://' + dhdwh_bucket + '/vid_fix_subscriptions_vw/1.0/parquet/year={}/month={}/day={}'
in_file_4 = 'gs://' + dhdwh_bucket + '/vid_d_customer/1.0/parquet/year={}/month={}/day={}'
in_file_5 = 'gs://' + dhdwh_bucket + '/vid_d_circuit/1.0/parquet/year={}/month={}/day={}'

#in_file_6 = 'gs://'+ network_rawprepared_bucket +'/fixed_network_data/'+ version +'/parquet/year={}/month={}/'

in_file_7 = 'gs://' + dhdwh_bucket + '/fixed_sr_tt/1.0/parquet/year={}/month={}/'

in_file_8 = 'gs://'+ dhdwh_bucket + '/fixed_orders/1.0/parquet/year={}/month={}/'

In [14]:
subprocess.call('/bin/sh /usr/bin/gsutil -q cp gs://' + files_bucket + '/notebooks/jupyter/home_lac/ENUTS3.csv ENUTS3.csv', shell=True)

0

# Cable providers

In [15]:
subprocess.call('/bin/sh /usr/bin/gsutil -q cp gs://' + files_bucket + '/notebooks/jupyter/fixed/cable_providers_Dec22.csv cable_providers_Dec22.csv', shell=True)
cable_providers = pd.read_csv('cable_providers_Dec22.csv')

0

In [16]:
cable_providers = cable_providers[['CO_code', 'kvID', "City", "County", "Longtitude", "Latitude", "Provider", "VDSL50", "VDSL100",
                      "VDSL200", "Connections", "Length", "CO_kv_Dist", "Type"]]

In [17]:
cable_providers['CO_code'] = cable_providers['CO_code'].astype(str)
cable_providers['Longtitude'] = cable_providers['Longtitude'].astype(float)
cable_providers['Latitude'] = cable_providers['Latitude'].astype(float)

In [18]:
# Null values
cable_providers.isnull().values.any()
for col in list(cable_providers.columns.values):
    if cable_providers[col].isnull().sum() > 0:
        print(col , ':' , cable_providers[col].isnull().sum())

True

City : 17
Connections : 42
Length : 42
CO_kv_Dist : 42
Type : 42


In [19]:
cable_providers.shape

(23642, 14)

In [20]:
cable_providers = cable_providers.drop_duplicates()

In [21]:
cable_providers.shape

(23635, 14)

In [22]:
cable_providers["VDSL_SUM"] = cable_providers["VDSL50"] + cable_providers["VDSL100"] + cable_providers["VDSL200"]

In [23]:
# Duplicates (CO_code, kvID)
dupl_cable_prov = cable_providers[cable_providers.duplicated(subset = ['CO_code', 'kvID'], keep = False)]
dupl_cable_prov.shape

(30, 15)

In [24]:
dupl_cable_prov = dupl_cable_prov.sort_values('VDSL_SUM',ascending=False)

In [25]:
cable_providers = cable_providers.sort_values('VDSL_SUM',ascending=False)
cable_providers = cable_providers.drop_duplicates(['CO_code', 'kvID'], keep = "first")

In [26]:
cable_providers.shape

(23620, 15)

In [27]:
cable_providers = cable_providers.sort_index()


In [28]:
cable_providers['CO_code'] = cable_providers['CO_code'].apply(lambda x:str(int(float(x))))

In [29]:
# Strange values
cable_providers[cable_providers['CO_code'] == '0.0'].shape
cable_providers[cable_providers['kvID'] == '0.0'].shape

cable_providers[cable_providers['CO_code'] == ''].shape
cable_providers[cable_providers['kvID'] == ''].shape

cable_providers[cable_providers['CO_code'] == '-'].shape
cable_providers[cable_providers['kvID'] == '-'].shape

(0, 15)

(0, 15)

(0, 15)

(0, 15)

(0, 15)

(0, 15)

In [30]:
# Duplicates (CO_code, kvID)
dupl_cable_prov = cable_providers[cable_providers.duplicated(subset = ['CO_code', 'kvID'], keep = False)]
dupl_cable_prov.shape

(0, 15)

In [31]:
kvid_coords = cable_providers[["CO_code", "kvID", "Longtitude", "Latitude"]]

In [32]:
kvid_coords

,CO_code,kvID,Longtitude,Latitude
0,440,103,23.737556,38.020639
1,440,104,23.737750,38.023333
2,440,105,23.738750,38.022389
3,440,107,23.741917,38.019917
4,440,109,23.742556,38.019972
...,...,...,...,...
23637,1149,4000,25.249729,35.227339
23638,1726,4000,23.133491,40.431839
23639,1317,311,25.121487,39.852649
23640,1953,4000,22.424835,39.395339


In [33]:
kvid_coords.shape

(23620, 4)

In [34]:
cable_providers.head(5)

In [36]:
cable_providers.groupby('CO_code')['kvID'].nunique()

CO_code
10        3
1001      2
1003      5
1004     15
1006      1
       ... 
991     143
992     149
993      21
995      35
996       6
Name: kvID, Length: 1248, dtype: int64

# CELLS

In [37]:
in_file_100 = 'gs://vfgr-dh-dwh-rawprepared/vid_ope_hubs_userinfo/1.0/parquet/year={}/month={}'

print(currentYear, currentMonth)
hubs = read_in_data_month(in_file_100, year = year_val[2], month = month_val[2]).select('day')
day = hubs.select(f.max(f.col("day")).alias("MAX")).limit(1).collect()[0].MAX
print(day)

2023 3
31


In [38]:
vid_ope_hubs_userinfo = read_in_data_day(in_file_1, year_val[2], month = month_val[2], day=day)

In [39]:
in_file_200 = 'gs://vfgr-dh-dwh-rawprepared/vid_ope_v_dslam_users/1.0/parquet/year={}/month={}'

print(currentYear, currentMonth)
dslam = read_in_data_month(in_file_200, year=year_val[2], month = month_val[2]).select('day')
day = dslam.select(f.max(f.col("day")).alias("MAX")).limit(1).collect()[0].MAX
print(day)

2023 3
31


In [40]:
vid_ope_v_dslam_users = read_in_data_day(in_file_2, year=year_val[2], month = month_val[2], day=day)

In [41]:
vid_ope_hubs_userinfo.createOrReplaceTempView("vid_ope_hubs_userinfo")
vid_ope_v_dslam_users.createOrReplaceTempView("vid_ope_v_dslam_users")

cells1 = spark.sql("""
                select A.G_LONGITUDE, A.G_LATITUDE, A.WCRM, A.TOB, A.BANDWIDTH, A.G_WALKINGDST, A.G_RADIUSDST,
                A.VDSLESTIMATION, A.KVID, B.USER_NUM, B.PHONE, B.BANDWIDTH as BANDWIDTH_1, B.ID_TOB, B.CIRCUIT_TYPE, B.APPLICATION_PRODUCT,
                B.VDSL_TECHNOLOGY, B.LDAP_ID, B.KVID as KVID_1
                from vid_ope_hubs_userinfo A
                inner join vid_ope_v_dslam_users B
                on A.USERID = B.USER_NUM
                --where A.USERID != '' or B.USER_NUM !=''
                             """) 

In [42]:
vid_ope_hubs_userinfo.createOrReplaceTempView("vid_ope_hubs_userinfo")
vid_ope_v_dslam_users.createOrReplaceTempView("vid_ope_v_dslam_users")

cells2 = spark.sql("""
                select A.G_LONGITUDE, A.G_LATITUDE, A.WCRM, A.TOB, A.BANDWIDTH, A.G_WALKINGDST, A.G_RADIUSDST,
                A.VDSLESTIMATION, A.KVID, B.USER_NUM, B.PHONE, B.BANDWIDTH as BANDWIDTH_1, B.ID_TOB, B.CIRCUIT_TYPE, B.APPLICATION_PRODUCT,
                B.VDSL_TECHNOLOGY, B.LDAP_ID, B.KVID as KVID_1
                from vid_ope_hubs_userinfo A
                inner join vid_ope_v_dslam_users B
                on A.TOB = B.ID_TOB
                where (A.TOB !='' and B.ID_TOB !='')
                and (A.TOB !='-' and B.ID_TOB !='-')
                and (A.TOB !='NA' and B.ID_TOB !='NA')
                and (A.TOB !='0' and B.ID_TOB !='0')
                and (A.TOB !='000' and B.ID_TOB !='000')
                and (A.TOB !='0000' and B.ID_TOB !='0000')                
                             """)  

In [43]:
cells = cells1.union(cells2)
#cells.count()
# Handle duplicates
cells = cells.drop_duplicates()
#cells.count()

In [44]:
cells = cells.toPandas()

In [45]:
# Convert some columns
cells['PHONE'] = cells['PHONE'].str[3:]
cells['G_LONGITUDE'] = cells['G_LONGITUDE'].astype(float)
cells['G_LATITUDE'] = cells['G_LATITUDE'].astype(float)

In [46]:
# Check zero / negative values for longitude, latitude
cells['Neg_Latlong_Flag'] = np.where( (cells['G_LONGITUDE'] <= 0.000000) | (cells['G_LATITUDE'] <= 0.000000),0, 1)
cells['Neg_Latlong_Flag'].value_counts()

1    834751
0     44664
Name: Neg_Latlong_Flag, dtype: int64

In [47]:
# Duplicates
cells[cells.duplicated(keep=False)].shape
cells = cells.drop_duplicates()
cells.shape

(0, 19)

(879415, 19)

In [48]:
# Duplicates (ldapid)
dupl_cell_ldapid = cells[cells.duplicated(subset = "LDAP_ID", keep = False)]
dupl_cell_ldapid.shape
#dupl_cell_ldapid.sort_values("LDAP_ID", ascending = False).head()

(2247, 19)

In [49]:
cells[cells.duplicated(subset = "LDAP_ID", keep = False)].shape

(2247, 19)

# MAP & CELLS

In [50]:
cells[cells.duplicated(subset = ['G_LATITUDE','G_LONGITUDE'], keep=False)].shape


(491979, 19)

In [51]:
unique_coord_cells = cells[['G_LATITUDE','G_LONGITUDE']].drop_duplicates()

In [52]:
# Create a geopandas dataframe from latlong info of customers
import geopandas as gpd
geo_cells = gpd.GeoDataFrame(unique_coord_cells, geometry=gpd.points_from_xy(unique_coord_cells.G_LONGITUDE, unique_coord_cells.G_LATITUDE))

In [53]:
# Import geopandas for Greece map
import subprocess
subprocess.call('/bin/sh /usr/bin/gsutil -q cp gs://vf-gr-ca-nonlive-devde/notebooks/jupyter/home_lac/ENUTS3.csv ENUTS3.csv', shell=True)

0

In [54]:
from shapely import wkt
geo_map = pd.read_csv('ENUTS3.csv')
geo_map['geometry'] = geo_map['geometry'].apply(wkt.loads)
geo_map = gpd.GeoDataFrame(geo_map, crs='epsg:4326')
# select Greece
geo_gr = geo_map[geo_map.CNTRY_NAME == 'Greece']

In [55]:
# Join global map with cells
geo_map_cells = gpd.sjoin(geo_cells, geo_gr[['ADMIN_NAME', 'geometry']], how='left', op='within')

/opt/conda/anaconda/lib/python3.6/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [56]:
hubs = cells.merge(geo_map_cells, on =['G_LATITUDE','G_LONGITUDE'])
hubs.shape

(879415, 22)

In [57]:
# I delete rows with null LDAPID !!
hubs[hubs["LDAP_ID"] == ''].shape
hubs = hubs[hubs['LDAP_ID'] != '']
hubs.shape

(29, 22)

(879386, 22)

In [58]:
# Handle values for KVID
# Filla nans of KVID with 0 -> ADSL cases ?
hubs['KVID'] = hubs['KVID'].fillna('0.0')
hubs[hubs['KVID'] == '0.0'].shape, hubs[hubs['KVID_1'] == '0.0'].shape

hubs['KVID'] = hubs['KVID'].apply(lambda x:''.join([s for s in x if s.isdigit()]) if bool(re.search('[α-ωΑ-Ω]', str(x) )) else x)
hubs['KVID_1'] = hubs['KVID_1'].apply(lambda x:''.join([s for s in x if s.isdigit()]) if bool(re.search('[α-ωΑ-Ω]', str(x) )) else x)

hubs['KVID'] = hubs['KVID'].replace({'':'0','-':'0'})
hubs['KVID_1'] = hubs['KVID_1'].replace({'':'0','-':'0'})
#hubs[hubs['KVID'] == '0'].shape, hubs[hubs['KVID_1'] == '0'].shape

hubs['KVID'] = hubs['KVID'].astype(float).astype(int)
hubs['KVID_1'] = hubs['KVID_1'].astype(float).astype(int)

hubs['KVID'] = hubs['KVID'].apply(lambda x:str(int(float(x))) if x else '0')
hubs['KVID_1'] = hubs['KVID_1'].apply(lambda x:str(int(float(x))) if x else '0')

#hubs[hubs['KVID'] == '0.0'].shape, hubs[hubs['KVID_1'] == '0.0'].shape
hubs[hubs['KVID'] == '0'].shape, hubs[hubs['KVID_1'] == '0'].shape

((0, 22), (0, 22))

((46622, 22), (538404, 22))

In [59]:
# Handle null vales for kvID
hubs[(hubs.KVID == '0') & (hubs['KVID_1'] != '0')].shape
# if kvID is null -> replace it with the other kvID
hubs['KVID'] = np.where((hubs.KVID == '0') & (hubs['KVID_1'] != '0'), hubs['KVID_1'], hubs['KVID'])

(1630, 22)

In [60]:
# Duplicates
hubs.shape
hubs.drop_duplicates(subset=['TOB','G_LATITUDE','G_LONGITUDE']).shape, hubs.drop_duplicates(subset=['LDAP_ID','G_LATITUDE','G_LONGITUDE']).shape

(879386, 22)

((867073, 22), (879010, 22))

In [61]:
unique_hubs = hubs.drop_duplicates(subset=['LDAP_ID','G_LATITUDE','G_LONGITUDE'])

In [62]:
# Duplicates (CO_code, kvID)
dupl_hubs_ldapid = unique_hubs[unique_hubs.duplicated(subset = "LDAP_ID", keep = False)]
dupl_hubs_ldapid.shape
#dupl_hubs_ldapid.sort_values("LDAP_ID", ascending = False).head()

(1470, 22)

# FIXED BASE

In [63]:
in_file_300 = 'gs://vfgr-dh-dwh-rawprepared/vid_fix_subscriptions_vw/1.0/parquet/year={}/month={}'

print(currentYear, currentMonth)
subs = read_in_data_month(in_file_300, year=year_val[2], month = month_val[2]).select('day')
day_sub = subs.select(f.max(f.col("day")).alias("MAX")).limit(1).collect()[0].MAX
print(day_sub)

2023 3
31


In [64]:
in_file_400 = 'gs://vfgr-dh-dwh-rawprepared/vid_d_customer/1.0/parquet/year={}/month={}'

print(currentYear, currentMonth)
custs = read_in_data_month(in_file_400, year=year_val[2], month = month_val[2]).select('day')
day_cust = custs.select(f.max(f.col("day")).alias("MAX")).limit(1).collect()[0].MAX
print(day_cust)

2023 3
31


In [65]:
in_file_500 = 'gs://vfgr-dh-dwh-rawprepared/vid_d_circuit/1.0/parquet/year={}/month={}'

print(year_val[2], month_val[2])
circ = read_in_data_month(in_file_500, year=year_val[2], month = month_val[2]).select('day')
day_circ = circ.select(f.max(f.col("day")).alias("MAX")).limit(1).collect()[0].MAX
print(day_circ)

2022 12
31


In [66]:
vid_fix_subscriptions_vw = read_in_data_day(in_file_3, year=year_val[2], month = month_val[2], day=day_sub)
vid_d_customer = read_in_data_day(in_file_4, year=year_val[2], month = month_val[2], day=day_cust)
vid_d_circuit = read_in_data_day(in_file_5, year=year_val[2], month = month_val[2], day=day_circ)

In [67]:
vid_fix_subscriptions_vw.createOrReplaceTempView("vid_fix_subscriptions_vw")
vid_d_customer.createOrReplaceTempView("vid_d_customer")
vid_d_circuit.createOrReplaceTempView("vid_d_circuit")


fixed_base = spark.sql("""
            SELECT
            C.*, Q.LDAPID, Q.LOOPID, Q.LOOPSITE  FROM
            (SELECT
            B.CLI, B.SUBSCR_ID, A.CUST_ID, A.VF_SEGMENT,A.ZIP, A.CUST_STATUS_TYPE_CD,
            B.PRODUCT_NAME, B.SUBSCR_STATUS_CD,B.VDSL_TYPE, B.SPEED,B.AK_ID,B.KVID,
            B.SUBSCR_EFF_DATE,B.SUBSCR_RENEWAL_DTTM,B.CONTRACT_START_DT, B.CNTRCT_END_DATE
            FROM vid_d_customer A
            inner join vid_fix_subscriptions_vw B 
            ON A.CUST_ID = B.CUST_ID
            where B.PRODUCT_NAME IN ('HOL Double Play' , 'HOL Telephony (POTS)' , 'HOL Triple Play','HOL Office Double Play',  'HOL Business Double Play')
            and B.CLI is not null
            and A.CUST_ID is not null and B.CUST_ID is not null
            AND A.VF_SEGMENT NOT IN ('VGE','MAJOR ACCOUNT','HVSME')
            and B.SUBSCR_ID is not null
            AND A.CUST_STATUS_TYPE_CD != 'I'
            AND B.SUBSCR_STATUS_CD != 'I') C
            inner join (
            select LDAPID, SUBSCR_ID, LOOPID, LOOPSITE, WCRM_AK_ID,CRAMER_AK_ID from vid_d_circuit
            where SUBSCR_ID is not null
            and VALID_UNTIL_DT = '9999-12-31 00:00:00') Q
            on q.SUBSCR_ID = C.SUBSCR_ID
            """) 

In [68]:
fixed_base = fixed_base.drop("LOOPSITE")

In [69]:
# Convert all datasets to pandas
fixed_base = fixed_base["CLI", "LDAPID", "SUBSCR_RENEWAL_DTTM", "CONTRACT_START_DT", "SPEED", "VF_SEGMENT", "VDSL_TYPE"].toPandas()

In [70]:
# Duplicates
fixed_base[fixed_base.duplicated(keep = False)].shape
fixed_base = fixed_base.drop_duplicates()

(0, 7)

In [71]:
# I want to exclude rows with NaN CLIs
fixed_base = fixed_base.dropna(subset = ['CLI'])
fixed_base.shape

(827479, 7)

In [72]:
fixed_base['VF_SEGMENT'] = fixed_base['VF_SEGMENT'].fillna('N/A')
fixed_base['VF_SEGMENT'] = fixed_base['VF_SEGMENT'].replace(np.nan, 'N/A')
fixed_base['VF_SEGMENT'] = np.where(fixed_base['VF_SEGMENT'] == '', 'N/A', fixed_base['VF_SEGMENT'])

In [73]:
fixed_base = fixed_base.sort_values(by=['CLI','SUBSCR_RENEWAL_DTTM','CONTRACT_START_DT'],\
                                ascending=[True,False,False]).drop_duplicates(subset='CLI', keep='first')

In [74]:
fixed_base_new = fixed_base[fixed_base['LDAPID'] != '']

# FIXED NETWORK

## 1

In [75]:
# Read network data
network_rawprepared_bucket = "vfgr-dh-network-rawprepared/"
version = "1.0"
in_file_600 = 'gs://'+ network_rawprepared_bucket +'/fixed_network_data/'+ version +'/parquet/year={}/month={}/'

# 1 previous month
fixed_network_1mon_prev_alone = read_in_data_month(in_file_600, year=year_val[2], month=month_val[2]).drop('service_file_id')
# 2 previous month
fixed_network_2mon_prev_alone = read_in_data_month(in_file_600, year=year_val[3], month=month_val[3]).drop('service_file_id')
# 3 previous month
fixed_network_3mon_prev_alone = read_in_data_month(in_file_600, year=year_val[4], month=month_val[4]).drop('service_file_id')

In [76]:
fixed_network_1mon_prev_alone = fixed_network_1mon_prev_alone[fixed_network_1mon_prev_alone['RESTRICTEDUSERS'] != 1]

In [77]:
fixed_network_1mon_prev_alone = fixed_network_1mon_prev_alone.selectExpr('AAA as LDAP_ID', 'DT',
                         'ATTAINABLERATEDOWNCPE', 'ATTAINABLERATEDOWNDSLAM',
                         'ATTENUATIONDOWNCPE', 'ATTENUATIONDOWNDSLAM',
                         'DATARATEDOWN','DATARATEDOWNRADIUS','SYNCHRONIZATIONRATEDOWNDSLAM', 'SYNCHRONIZATIONRATEDOWNCPE',
                         
#                        'NOISEMARGINDOWNCPE', 
#                        'NOISEMARGINUPCPE', 
#                        'SHOWTIMERESTARTS',                      
                        'DISCCAUSEADMINRESET',
                        'DISCCAUSEHOSTREQUEST',
                        'DISCCAUSELOSTCARRIER',
                        'DISCCAUSELOSTSERVICE',
                        'DISCCAUSENASERROR',
                        'DISCCAUSENASREQUEST',
                        'DISCCAUSEPORTERROR',
                        'DISCCAUSESESSIONTIMEOUT',
                        'DISCCAUSEUSERERROR',
                        'DISCCAUSEUSERREQUEST',                                                 
                        
                        'DISCONNECTIONS',
                                                   
                        'INSESSIONTIME')

In [78]:
fixed_network_1mon_prev = fixed_network_1mon_prev_alone

In [79]:
# (Repeat steps for previous month data)

# 1. Speed
fixed_network_1mon_prev = fixed_network_1mon_prev.withColumn('Speed1',f.when(fixed_network_1mon_prev.DATARATEDOWNRADIUS.isNull(),fixed_network_1mon_prev.SYNCHRONIZATIONRATEDOWNDSLAM).otherwise(fixed_network_1mon_prev.DATARATEDOWNRADIUS))
fixed_network_1mon_prev = fixed_network_1mon_prev.withColumn('Speed2',f.when(fixed_network_1mon_prev.Speed1.isNull(),fixed_network_1mon_prev.SYNCHRONIZATIONRATEDOWNCPE).otherwise(fixed_network_1mon_prev.Speed1))
fixed_network_1mon_prev = fixed_network_1mon_prev.withColumn('Actual_Speed',f.when(fixed_network_1mon_prev.Speed2.isNull(),fixed_network_1mon_prev.DATARATEDOWN).otherwise(fixed_network_1mon_prev.Speed2))

cols = ("Speed1","Speed2")
fixed_network_1mon_prev = fixed_network_1mon_prev.drop(*cols)

# 2. Attainable Rate
fixed_network_1mon_prev = fixed_network_1mon_prev.withColumn('AttainableRate',f.when(fixed_network_1mon_prev.ATTAINABLERATEDOWNDSLAM.isNull(),fixed_network_1mon_prev.ATTAINABLERATEDOWNCPE)\
                       .otherwise(fixed_network_1mon_prev.ATTAINABLERATEDOWNDSLAM))

# 3. Attenuation
fixed_network_1mon_prev = fixed_network_1mon_prev.withColumn('Attenuation',f.when(fixed_network_1mon_prev.ATTENUATIONDOWNDSLAM.isNull(),fixed_network_1mon_prev.ATTENUATIONDOWNCPE)\
                       .otherwise(fixed_network_1mon_prev.ATTENUATIONDOWNDSLAM))

# Calculate dominant speed value
mode_speed = fixed_network_1mon_prev.groupBy(['LDAP_ID','Actual_Speed']).agg(f.count('Actual_Speed').alias('speed_occurencies'))
mode_speed = mode_speed.groupBy(['LDAP_ID']).agg(f.max(f.struct(f.col('speed_occurencies'),
                                                               f.col('Actual_Speed'))).alias('max')).select(f.col('LDAP_ID'),f.col('max.Actual_Speed').alias('1M_MODE_SPEED'))
fixed_network_1mon_prev = fixed_network_1mon_prev.join(mode_speed, on = 'LDAP_ID', how = 'left')


# 4. Overview
fixed_network_1M_overview = fixed_network_1mon_prev.groupby('LDAP_ID', '1M_MODE_SPEED')\
                            .agg(f.max('Actual_Speed').alias('1M_Max_Actual_Speed'), 
                            f.min('Actual_Speed').alias('1M_Min_Actual_Speed'), 
                            f.avg('Actual_Speed').alias('1M_Actual_Speed'),
                            f.avg('AttainableRate').alias('1M_AttainableRate'),
                            f.avg('Attenuation').alias('1M_Attenuation'),
                            f.sum('DISCONNECTIONS').alias('1M_ttl_disconnections'),

#                            f.avg('NOISEMARGINDOWNCPE').alias('1M_NOISEMARGINDOWNCPE'),
#                            f.avg('NOISEMARGINUPCPE').alias('1M_NOISEMARGINUPCPE'),                                        
#                            f.sum('SHOWTIMERESTARTS').alias('1M_ttl_ShowTimeRestarts'),
                            f.sum('DISCCAUSEADMINRESET').alias('1M_ttl_AdminResets'),
                            f.sum('DISCCAUSEHOSTREQUEST').alias('1M_ttl_HostRequest'),
                            f.sum('DISCCAUSELOSTCARRIER').alias('1M_ttl_LostCarrier'),
                            f.sum('DISCCAUSELOSTSERVICE').alias('1M_ttl_LostService'),
                            f.sum('DISCCAUSENASERROR').alias('1M_ttl_NasError'),
                            f.sum('DISCCAUSENASREQUEST').alias('1M_ttl_NasRequest'),
                            f.sum('DISCCAUSEPORTERROR').alias('1M_ttl_PortError'),
                            f.sum('DISCCAUSESESSIONTIMEOUT').alias('1M_ttl_SessionTimeOut'),
                            f.sum('DISCCAUSEUSERERROR').alias('1M_ttl_UserError'),
                            f.sum('DISCCAUSEUSERREQUEST').alias('1M_ttl_UserRequest'),                                                    
                            f.sum('INSESSIONTIME').alias('1M_ttl_INSESSIONTIME'),
                            f.avg('INSESSIONTIME').alias('1M_INSESSIONTIME'))

# 5. Convert to pandas
fixed_network_1M_overview = fixed_network_1M_overview.toPandas()

#fixed_network_1M_overview.rename(columns={'AAA':'LDAP_ID'}, inplace=True)

## 2

In [80]:
fixed_network_2mon_prev_alone = fixed_network_2mon_prev_alone[fixed_network_2mon_prev_alone['RESTRICTEDUSERS'] != 1]

In [81]:
fixed_network_2mon_prev_alone = fixed_network_2mon_prev_alone.selectExpr('AAA as LDAP_ID', 'DT',
                         'ATTAINABLERATEDOWNCPE', 'ATTAINABLERATEDOWNDSLAM',
                         'ATTENUATIONDOWNCPE', 'ATTENUATIONDOWNDSLAM',
                         'DATARATEDOWN','DATARATEDOWNRADIUS','SYNCHRONIZATIONRATEDOWNDSLAM', 'SYNCHRONIZATIONRATEDOWNCPE',
                         
#                        'NOISEMARGINDOWNCPE', 
#                        'NOISEMARGINUPCPE', 
#                        'SHOWTIMERESTARTS',                      
                        'DISCCAUSEADMINRESET',
                        'DISCCAUSEHOSTREQUEST',
                        'DISCCAUSELOSTCARRIER',
                        'DISCCAUSELOSTSERVICE',
                        'DISCCAUSENASERROR',
                        'DISCCAUSENASREQUEST',
                        'DISCCAUSEPORTERROR',
                        'DISCCAUSESESSIONTIMEOUT',
                        'DISCCAUSEUSERERROR',
                        'DISCCAUSEUSERREQUEST',                                                 
                        
                        'DISCONNECTIONS',
                                                   
                        'INSESSIONTIME')

In [82]:
fixed_network_2mon_prev = fixed_network_2mon_prev_alone
#fixed_network_2mon_prev = fixed_network_1mon_prev_alone.union(fixed_network_2mon_prev_alone)

In [83]:
# (Repeat steps for previous month data)

# 1. Speed
fixed_network_2mon_prev = fixed_network_2mon_prev.withColumn('Speed1',f.when(fixed_network_2mon_prev.DATARATEDOWNRADIUS.isNull(),fixed_network_2mon_prev.SYNCHRONIZATIONRATEDOWNDSLAM).otherwise(fixed_network_2mon_prev.DATARATEDOWNRADIUS))
fixed_network_2mon_prev = fixed_network_2mon_prev.withColumn('Speed2',f.when(fixed_network_2mon_prev.Speed1.isNull(),fixed_network_2mon_prev.SYNCHRONIZATIONRATEDOWNCPE).otherwise(fixed_network_2mon_prev.Speed1))
fixed_network_2mon_prev = fixed_network_2mon_prev.withColumn('Actual_Speed',f.when(fixed_network_2mon_prev.Speed2.isNull(),fixed_network_2mon_prev.DATARATEDOWN).otherwise(fixed_network_2mon_prev.Speed2))

fixed_network_2mon_prev = fixed_network_2mon_prev.drop(*cols)

# 2. Attainable Rate
fixed_network_2mon_prev = fixed_network_2mon_prev.withColumn('AttainableRate',f.when(fixed_network_2mon_prev.ATTAINABLERATEDOWNDSLAM.isNull(),fixed_network_2mon_prev.ATTAINABLERATEDOWNCPE)\
                       .otherwise(fixed_network_2mon_prev.ATTAINABLERATEDOWNDSLAM))

# 3. Attenuation
fixed_network_2mon_prev = fixed_network_2mon_prev.withColumn('Attenuation',f.when(fixed_network_2mon_prev.ATTENUATIONDOWNDSLAM.isNull(),fixed_network_2mon_prev.ATTENUATIONDOWNCPE)\
                       .otherwise(fixed_network_2mon_prev.ATTENUATIONDOWNDSLAM))

# Calculate dominant speed value
mode_speed = fixed_network_2mon_prev.groupBy(['LDAP_ID','Actual_Speed']).agg(f.count('Actual_Speed').alias('speed_occurencies'))
mode_speed = mode_speed.groupBy(['LDAP_ID']).agg(f.max(f.struct(f.col('speed_occurencies'),
                                                               f.col('Actual_Speed'))).alias('max')).select(f.col('LDAP_ID'),f.col('max.Actual_Speed').alias('2M_MODE_SPEED'))
fixed_network_2mon_prev = fixed_network_2mon_prev.join(mode_speed, on = 'LDAP_ID', how = 'left')

# 4. Overview
fixed_network_2M_overview = fixed_network_2mon_prev.groupby('LDAP_ID', '2M_MODE_SPEED')\
                            .agg(f.max('Actual_Speed').alias('2M_Max_Actual_Speed'), 
                            f.min('Actual_Speed').alias('2M_Min_Actual_Speed'), 
                            f.avg('Actual_Speed').alias('2M_Actual_Speed'),
                            f.avg('AttainableRate').alias('2M_AttainableRate'),
                            f.avg('Attenuation').alias('2M_Attenuation'),
                            f.sum('DISCONNECTIONS').alias('2M_ttl_disconnections'),

#                            f.avg('NOISEMARGINDOWNCPE').alias('2M_NOISEMARGINDOWNCPE'),
#                            f.avg('NOISEMARGINUPCPE').alias('2M_NOISEMARGINUPCPE'),                                        
#                            f.sum('SHOWTIMERESTARTS').alias('2M_ttl_ShowTimeRestarts'),
                            f.sum('DISCCAUSEADMINRESET').alias('2M_ttl_AdminResets'),
                            f.sum('DISCCAUSEHOSTREQUEST').alias('2M_ttl_HostRequest'),
                            f.sum('DISCCAUSELOSTCARRIER').alias('2M_ttl_LostCarrier'),
                            f.sum('DISCCAUSELOSTSERVICE').alias('2M_ttl_LostService'),
                            f.sum('DISCCAUSENASERROR').alias('2M_ttl_NasError'),
                            f.sum('DISCCAUSENASREQUEST').alias('2M_ttl_NasRequest'),
                            f.sum('DISCCAUSEPORTERROR').alias('2M_ttl_PortError'),
                            f.sum('DISCCAUSESESSIONTIMEOUT').alias('2M_ttl_SessionTimeOut'),
                            f.sum('DISCCAUSEUSERERROR').alias('2M_ttl_UserError'),
                            f.sum('DISCCAUSEUSERREQUEST').alias('2M_ttl_UserRequest'),                                                    
                            f.sum('INSESSIONTIME').alias('2M_ttl_INSESSIONTIME'),
                            f.avg('INSESSIONTIME').alias('2M_INSESSIONTIME'))

# 5. Convert to pandas
fixed_network_2M_overview = fixed_network_2M_overview.toPandas()

#fixed_network_2M_overview.rename(columns={'AAA':'LDAP_ID'}, inplace=True)

## 3

In [84]:
fixed_network_3mon_prev_alone = fixed_network_3mon_prev_alone[fixed_network_3mon_prev_alone['RESTRICTEDUSERS'] != 1]

In [85]:
fixed_network_3mon_prev_alone = fixed_network_3mon_prev_alone.selectExpr('AAA as LDAP_ID', 'DT',
                         'ATTAINABLERATEDOWNCPE', 'ATTAINABLERATEDOWNDSLAM',
                         'ATTENUATIONDOWNCPE', 'ATTENUATIONDOWNDSLAM',
                         'DATARATEDOWN','DATARATEDOWNRADIUS','SYNCHRONIZATIONRATEDOWNDSLAM', 'SYNCHRONIZATIONRATEDOWNCPE',
                         
#                        'NOISEMARGINDOWNCPE', 
#                        'NOISEMARGINUPCPE', 
#                        'SHOWTIMERESTARTS',                      
                        'DISCCAUSEADMINRESET',
                        'DISCCAUSEHOSTREQUEST',
                        'DISCCAUSELOSTCARRIER',
                        'DISCCAUSELOSTSERVICE',
                        'DISCCAUSENASERROR',
                        'DISCCAUSENASREQUEST',
                        'DISCCAUSEPORTERROR',
                        'DISCCAUSESESSIONTIMEOUT',
                        'DISCCAUSEUSERERROR',
                        'DISCCAUSEUSERREQUEST',                                                 
                        
                        'DISCONNECTIONS',
                                                   
                        'INSESSIONTIME')

In [86]:
fixed_network_3mon_prev =  fixed_network_3mon_prev_alone
#fixed_network_3mon_prev =  fixed_network_3mon_prev_alone.union(fixed_network_2mon_prev_alone).union(fixed_network_1mon_prev_alone)

In [87]:
# (Repeat steps for previous month data)

# 1. Speed
fixed_network_3mon_prev = fixed_network_3mon_prev.withColumn('Speed1',f.when(fixed_network_3mon_prev.DATARATEDOWNRADIUS.isNull(),fixed_network_3mon_prev.SYNCHRONIZATIONRATEDOWNDSLAM).otherwise(fixed_network_3mon_prev.DATARATEDOWNRADIUS))
fixed_network_3mon_prev = fixed_network_3mon_prev.withColumn('Speed2',f.when(fixed_network_3mon_prev.Speed1.isNull(),fixed_network_3mon_prev.SYNCHRONIZATIONRATEDOWNCPE).otherwise(fixed_network_3mon_prev.Speed1))
fixed_network_3mon_prev = fixed_network_3mon_prev.withColumn('Actual_Speed',f.when(fixed_network_3mon_prev.Speed2.isNull(),fixed_network_3mon_prev.DATARATEDOWN).otherwise(fixed_network_3mon_prev.Speed2))

fixed_network_3mon_prev = fixed_network_3mon_prev.drop(*cols)

# 2. Attainable Rate
fixed_network_3mon_prev = fixed_network_3mon_prev.withColumn('AttainableRate',f.when(fixed_network_3mon_prev.ATTAINABLERATEDOWNDSLAM.isNull(),fixed_network_3mon_prev.ATTAINABLERATEDOWNCPE)\
                       .otherwise(fixed_network_3mon_prev.ATTAINABLERATEDOWNDSLAM))

# 3. Attenuation
fixed_network_3mon_prev = fixed_network_3mon_prev.withColumn('Attenuation',f.when(fixed_network_3mon_prev.ATTENUATIONDOWNDSLAM.isNull(),fixed_network_3mon_prev.ATTENUATIONDOWNCPE)\
                       .otherwise(fixed_network_3mon_prev.ATTENUATIONDOWNDSLAM))

# Calculate dominant speed value
mode_speed = fixed_network_3mon_prev.groupBy(['LDAP_ID','Actual_Speed']).agg(f.count('Actual_Speed').alias('speed_occurencies'))
mode_speed = mode_speed.groupBy(['LDAP_ID']).agg(f.max(f.struct(f.col('speed_occurencies'),
                                                               f.col('Actual_Speed'))).alias('max')).select(f.col('LDAP_ID'),f.col('max.Actual_Speed').alias('3M_MODE_SPEED'))
fixed_network_3mon_prev = fixed_network_3mon_prev.join(mode_speed, on = 'LDAP_ID', how = 'left')

# 4. Overview
fixed_network_3M_overview = fixed_network_3mon_prev.groupby('LDAP_ID', '3M_MODE_SPEED')\
                            .agg(f.max('Actual_Speed').alias('3M_Max_Actual_Speed'), 
                            f.min('Actual_Speed').alias('3M_Min_Actual_Speed'), 
                            f.avg('Actual_Speed').alias('3M_Actual_Speed'),
                            f.avg('AttainableRate').alias('3M_AttainableRate'),
                            f.avg('Attenuation').alias('3M_Attenuation'),
                            f.sum('DISCONNECTIONS').alias('3M_ttl_disconnections'),                                 
#                            f.avg('NOISEMARGINDOWNCPE').alias('3M_NOISEMARGINDOWNCPE'),
#                            f.avg('NOISEMARGINUPCPE').alias('3M_NOISEMARGINUPCPE'),                                        
#                            f.sum('SHOWTIMERESTARTS').alias('3M_ttl_ShowTimeRestarts'),
                            f.sum('DISCCAUSEADMINRESET').alias('3M_ttl_AdminResets'),
                            f.sum('DISCCAUSEHOSTREQUEST').alias('3M_ttl_HostRequest'),
                            f.sum('DISCCAUSELOSTCARRIER').alias('3M_ttl_LostCarrier'),
                            f.sum('DISCCAUSELOSTSERVICE').alias('3M_ttl_LostService'),
                            f.sum('DISCCAUSENASERROR').alias('3M_ttl_NasError'),
                            f.sum('DISCCAUSENASREQUEST').alias('3M_ttl_NasRequest'),
                            f.sum('DISCCAUSEPORTERROR').alias('3M_ttl_PortError'),
                            f.sum('DISCCAUSESESSIONTIMEOUT').alias('3M_ttl_SessionTimeOut'),
                            f.sum('DISCCAUSEUSERERROR').alias('3M_ttl_UserError'),
                            f.sum('DISCCAUSEUSERREQUEST').alias('3M_ttl_UserRequest'),                                                    
                            f.sum('INSESSIONTIME').alias('3M_ttl_INSESSIONTIME'),
                            f.avg('INSESSIONTIME').alias('3M_INSESSIONTIME'))

# 5. Convert to pandas
fixed_network_3M_overview = fixed_network_3M_overview.toPandas()

#fixed_network_3M_overview.rename(columns={'AAA':'LDAP_ID'}, inplace=True)

In [88]:
fixed_network_overview_1 = fixed_network_1M_overview.merge(fixed_network_2M_overview, on= 'LDAP_ID')
fixed_network_overview = fixed_network_overview_1.merge(fixed_network_3M_overview, on= 'LDAP_ID')

In [89]:
# # Modify Attenuation
#fixed_network_overview['1W_Attenuation_10'] = fixed_network_overview['1W_Attenuation']/10
fixed_network_overview['1M_Attenuation_10'] = fixed_network_overview['1M_Attenuation']/10
fixed_network_overview['2M_Attenuation_10'] = fixed_network_overview['2M_Attenuation']/10
fixed_network_overview['3M_Attenuation_10'] = fixed_network_overview['3M_Attenuation']/10

fixed_network_overview['1M_Attenuation_10'].max()
fixed_network_overview['1M_Attenuation_10'].mean()

# Modify insession time
# fixed_network_overview['1M_INSESSIONTIME'] = fixed_network_overview['1M_ttl_INSESSIONTIME'] / (int(day_max) * (24 * 60 * 60))
# fixed_network_overview['2M_INSESSIONTIME'] = fixed_network_overview['2M_ttl_INSESSIONTIME'] / (int(day_max) * (24 * 60 * 60))
# fixed_network_overview['3M_INSESSIONTIME'] = fixed_network_overview['3M_ttl_INSESSIONTIME'] / (int(day_max) * (24 * 60 * 60))

# Modify mode speed
fixed_network_overview['1M_MODE_SPEED_1000'] = fixed_network_overview['1M_MODE_SPEED']/1000
fixed_network_overview['2M_MODE_SPEED_1000'] = fixed_network_overview['2M_MODE_SPEED']/1000
fixed_network_overview['3M_MODE_SPEED_1000'] = fixed_network_overview['3M_MODE_SPEED']/1000

# Modify actual speed
#fixed_network_overview['1W_Actual_Speed_1000'] = fixed_network_overview['1W_Actual_Speed']/1000
fixed_network_overview['1M_Actual_Speed_1000'] = fixed_network_overview['1M_Actual_Speed']/1000
fixed_network_overview['2M_Actual_Speed_1000'] = fixed_network_overview['2M_Actual_Speed']/1000
fixed_network_overview['3M_Actual_Speed_1000'] = fixed_network_overview['3M_Actual_Speed']/1000


# Modify max actual speed
#fixed_network_overview['1W_Max_Actual_Speed_1000'] = fixed_network_overview['1W_Max_Actual_Speed']/1000
fixed_network_overview['1M_Max_Actual_Speed_1000'] = fixed_network_overview['1M_Max_Actual_Speed']/1000
fixed_network_overview['2M_Max_Actual_Speed_1000'] = fixed_network_overview['2M_Max_Actual_Speed']/1000
fixed_network_overview['3M_Max_Actual_Speed_1000'] = fixed_network_overview['3M_Max_Actual_Speed']/1000

# Modify min actual speed
#fixed_network_overview['1W_Min_Actual_Speed_1000'] = fixed_network_overview['1W_Min_Actual_Speed']/1000
fixed_network_overview['1M_Min_Actual_Speed_1000'] = fixed_network_overview['1M_Min_Actual_Speed']/1000
fixed_network_overview['2M_Min_Actual_Speed_1000'] = fixed_network_overview['2M_Min_Actual_Speed']/1000
fixed_network_overview['3M_Min_Actual_Speed_1000'] = fixed_network_overview['3M_Min_Actual_Speed']/1000

54.26548387096774

2.809543340515553

In [90]:
# Fill Null values for  1M attenuation
fixed_network_overview['1M_Attenuation_10'] = fixed_network_overview['1M_Attenuation_10'].fillna(fixed_network_overview['1M_Attenuation_10'].mean())
fixed_network_overview['1M_Attenuation_10'] = np.where(fixed_network_overview['1M_Attenuation_10'] == '',
                                                               fixed_network_overview['1M_Attenuation_10'].mean(),fixed_network_overview['1M_Attenuation_10'])
fixed_network_overview['1M_Attenuation_10'] = np.where(fixed_network_overview['1M_Attenuation_10'] == '-',
                                                               fixed_network_overview['1M_Attenuation_10'].mean(),fixed_network_overview['1M_Attenuation_10'])
fixed_network_overview['1M_Attenuation_10'] = fixed_network_overview['1M_Attenuation_10'].replace(np.nan,
                                                               fixed_network_overview['1M_Attenuation_10'].mean())

# Fill Null values for 2M attenuation
fixed_network_overview['2M_Attenuation_10'] = fixed_network_overview['2M_Attenuation_10'].fillna(fixed_network_overview['2M_Attenuation_10'].mean())
fixed_network_overview['2M_Attenuation_10'] = np.where( fixed_network_overview['2M_Attenuation_10'] == '',
                                                               fixed_network_overview['2M_Attenuation_10'].mean(),fixed_network_overview['2M_Attenuation_10'])
fixed_network_overview['2M_Attenuation_10'] = np.where( fixed_network_overview['2M_Attenuation_10'] == '-',
                                                               fixed_network_overview['2M_Attenuation_10'].mean(),fixed_network_overview['2M_Attenuation_10'])
fixed_network_overview['2M_Attenuation_10'] = fixed_network_overview['2M_Attenuation_10'].replace(np.nan,
                                                               fixed_network_overview['2M_Attenuation_10'].mean())

# Fill Null values for 3M attenuation
fixed_network_overview['3M_Attenuation_10'] = fixed_network_overview['3M_Attenuation_10'].fillna(fixed_network_overview['3M_Attenuation_10'].mean())
fixed_network_overview['3M_Attenuation_10'] = np.where( fixed_network_overview['3M_Attenuation_10'] == '',
                                                               fixed_network_overview['3M_Attenuation_10'].mean(),fixed_network_overview['3M_Attenuation_10'])
fixed_network_overview['3M_Attenuation_10'] = np.where( fixed_network_overview['3M_Attenuation_10'] == '-',
                                                               fixed_network_overview['3M_Attenuation_10'].mean(),fixed_network_overview['3M_Attenuation_10'])
fixed_network_overview['3M_Attenuation_10'] = fixed_network_overview['3M_Attenuation_10'].replace(np.nan,
                                                               fixed_network_overview['3M_Attenuation_10'].mean())

In [91]:
# Fill nulls with data from previous months
fixed_network_overview['3M_AttainableRate'] = np.where(fixed_network_overview['3M_AttainableRate'].isnull(), fixed_network_overview['2M_AttainableRate'], fixed_network_overview['3M_AttainableRate'])
fixed_network_overview['3M_AttainableRate'] = np.where(fixed_network_overview['3M_AttainableRate'].isnull(), fixed_network_overview['1M_AttainableRate'], fixed_network_overview['3M_AttainableRate'])
#fixed_network_overview['3M_AttainableRate'] = np.where(fixed_network_overview['3M_AttainableRate'].isnull(), fixed_network_overview['1W_AttainableRate'], fixed_network_overview['3M_AttainableRate'])

fixed_network_overview['2M_AttainableRate'] = np.where(fixed_network_overview['2M_AttainableRate'].isnull(), fixed_network_overview['3M_AttainableRate'], fixed_network_overview['2M_AttainableRate'])
fixed_network_overview['2M_AttainableRate'] = np.where(fixed_network_overview['2M_AttainableRate'].isnull(), fixed_network_overview['1M_AttainableRate'], fixed_network_overview['2M_AttainableRate'])
#fixed_network_overview['2M_AttainableRate'] = np.where(fixed_network_overview['2M_AttainableRate'].isnull(), fixed_network_overview['1W_AttainableRate'], fixed_network_overview['2M_AttainableRate'])

fixed_network_overview['1M_AttainableRate'] = np.where(fixed_network_overview['1M_AttainableRate'].isnull(), fixed_network_overview['3M_AttainableRate'], fixed_network_overview['1M_AttainableRate'])
fixed_network_overview['1M_AttainableRate'] = np.where(fixed_network_overview['1M_AttainableRate'].isnull(), fixed_network_overview['2M_AttainableRate'], fixed_network_overview['1M_AttainableRate'])
#fixed_network_overview['1M_AttainableRate'] = np.where(fixed_network_overview['1M_AttainableRate'].isnull(), fixed_network_overview['1W_AttainableRate'], fixed_network_overview['1M_AttainableRate'])

In [92]:
fixed_network_overview['ceil_1M_Attenuation_10'] = fixed_network_overview['1M_Attenuation_10'].apply(lambda x: math.ceil(x))
#fixed_network_overview['round_1M_Attenuation_10'] = fixed_network_overview['1M_Attenuation_10'].apply(lambda x: round(f.lit(x)))
#fixed_network_overview['round_1M_Attenuation_10'] = fixed_network_overview['1M_Attenuation_10'].apply(lambda x: round(x))

fixed_network_overview['ceil_2M_Attenuation_10'] = fixed_network_overview['2M_Attenuation_10'].apply(lambda x: math.ceil(x))
#fixed_network_overview['round_2M_Attenuation_10'] = fixed_network_overview['2M_Attenuation_10'].apply(lambda x: round(x))

fixed_network_overview['ceil_3M_Attenuation_10'] = fixed_network_overview['3M_Attenuation_10'].apply(lambda x: math.ceil(x))
#fixed_network_overview['round_3M_Attenuation_10'] = fixed_network_overview['3M_Attenuation_10'].apply(lambda x: round(x))

In [93]:
# # Average 3mhnou
fixed_network_overview['AVG_2M_MODE_SPEED'] =  (fixed_network_overview['1M_MODE_SPEED_1000'] + fixed_network_overview['2M_MODE_SPEED_1000']) / 2
fixed_network_overview['AVG_3M_MODE_SPEED'] =  (fixed_network_overview['1M_MODE_SPEED_1000'] + fixed_network_overview['2M_MODE_SPEED_1000'] + fixed_network_overview['3M_MODE_SPEED_1000']) / 3

fixed_network_overview['AVG_2M_Actual_Speed'] =  (fixed_network_overview['1M_Actual_Speed_1000'] + fixed_network_overview['2M_Actual_Speed_1000']) / 2
fixed_network_overview['AVG_3M_Actual_Speed'] =  (fixed_network_overview['1M_Actual_Speed_1000'] + fixed_network_overview['2M_Actual_Speed_1000'] + fixed_network_overview['3M_Actual_Speed_1000']) / 3

fixed_network_overview['AVG_2M_Attenuation'] =  (fixed_network_overview['ceil_1M_Attenuation_10'] + fixed_network_overview['ceil_2M_Attenuation_10']) / 2
fixed_network_overview['AVG_3M_Attenuation'] =  (fixed_network_overview['ceil_1M_Attenuation_10'] + fixed_network_overview['ceil_2M_Attenuation_10'] + fixed_network_overview['ceil_3M_Attenuation_10']) / 3

fixed_network_overview['AVG_2M_ttl_disconnections'] =  (fixed_network_overview['1M_ttl_disconnections'] + fixed_network_overview['2M_ttl_disconnections']) / 2
fixed_network_overview['AVG_3M_ttl_disconnections'] =  (fixed_network_overview['1M_ttl_disconnections'] + fixed_network_overview['2M_ttl_disconnections'] + fixed_network_overview['3M_ttl_disconnections']) / 3

In [94]:
print("!")

!


# Merge

In [95]:
# 1. Hubs & Fixed Base

In [96]:
unique_hubs.shape, fixed_base_new.shape
fixed_base_hubs = unique_hubs.merge(fixed_base_new, left_on='LDAP_ID', right_on='LDAPID')
fixed_base_hubs.shape

((879010, 22), (827251, 7))

(794307, 29)

In [97]:
fixed_base_hubs = fixed_base_hubs.rename(columns={'KVID':'kvID'})
fixed_base_hubs = fixed_base_hubs.rename(columns={'WCRM':'CO_code'})

In [98]:
# SOS: usually there are null values there so I have to fill nans for 
fixed_base_hubs['CO_code'] = fixed_base_hubs['CO_code'].fillna(0.0)
fixed_base_hubs['CO_code'] = np.where(fixed_base_hubs['CO_code'] == '', 0.0, fixed_base_hubs['CO_code'])
fixed_base_hubs['CO_code'] = fixed_base_hubs['CO_code'].replace(np.nan, 0.0)

In [99]:
fixed_base_hubs['CO_code'] = fixed_base_hubs['CO_code'].apply(lambda x:str(int(float(x))))

In [100]:
fixed_base_hubs['LDAP_ID'] = fixed_base_hubs['LDAP_ID'].astype(str)

In [101]:
def recalculate_dsl(x):
                if (x['SPEED'] == '24 Mbps') or (x['SPEED'] == '24 mbps') or (x['SPEED'] == '12 Mbps') or (x['SPEED'] == '2 Mbps'):
                                return 'ADSL'
                elif (x['SPEED'] == '30 Mbps'):
                                return '30'
                elif (x['SPEED'] == '50 Mbps'):
                                return '50'
                elif (x['SPEED'] == '100 Mbps') or (x['SPEED'] == '100 Mpbs'):
                                return '100'
                elif (x['SPEED'] == '200 Mbps'):
                                return '200'
                else:
                                return "N/A"

In [102]:
fixed_base_hubs['DSL'] = fixed_base_hubs.apply(recalculate_dsl, axis=1)

In [103]:
fixed_base_hubs['VDSL_TECHNOLOGY'] = fixed_base_hubs['VDSL_TECHNOLOGY'].replace(['ONNET'], 'VDSL OnNet')
fixed_base_hubs['VDSL_TECHNOLOGY'] = fixed_base_hubs['VDSL_TECHNOLOGY'].replace(['OFFNET'], 'VDSL OffNet')
fixed_base_hubs['VDSL_TECHNOLOGY'] = fixed_base_hubs['VDSL_TECHNOLOGY'].replace(['VF FTTH'], 'FTTH')

In [104]:
# Fill null values of VDSL Type with VDSL Tehnology
fixed_base_hubs['VDSL_TYPE'] = np.where(fixed_base_hubs['VDSL_TYPE'] == '', fixed_base_hubs['VDSL_TECHNOLOGY'], fixed_base_hubs['VDSL_TYPE'])

In [105]:
fixed_base_hubs[fixed_base_hubs.duplicated(subset = ['CO_code','kvID'], keep = False)].head(20)

,G_LONGITUDE,G_LATITUDE,CO_code,TOB,BANDWIDTH,G_WALKINGDST,G_RADIUSDST,VDSLESTIMATION,kvID,USER_NUM,PHONE,BANDWIDTH_1,ID_TOB,CIRCUIT_TYPE,APPLICATION_PRODUCT,VDSL_TECHNOLOGY,LDAP_ID,KVID_1,Neg_Latlong_Flag,geometry,index_right,ADMIN_NAME,CLI,LDAPID,SUBSCR_RENEWAL_DTTM,CONTRACT_START_DT,SPEED,VF_SEGMENT,VDSL_TYPE,DSL
0,22.962849,40.600790,1026,21Β1884869,30720/3072,475.284308,367.079091,0.0,469,101696921,2359243648,30/3,21Β1884869,FLLU,HOL Double Play,VDSL OnNet,643234,0,1,POINT (22.96285 40.60079),6.0,Thessaloniki,2359243648,643234,2022-12-13,2022-12-13,30 Mbps,RETAIL,VDSL OnNet,30
1,22.962849,40.600790,1026,21Β2576887,24576/1024,475.291342,367.079091,24986.0,469,104779182,2343990804,24/1,21Β2576887,FLLU,HOL Double Play,,1448348,0,1,POINT (22.96285 40.60079),6.0,Thessaloniki,2343990804,1448348,2021-07-14,2021-07-14,24 Mbps,N/A,,ADSL
2,22.962849,40.600790,1026,21Β3879307,24576/1024,475.291342,367.079091,28310.0,469,104779327,2141377472,24/1,21Β3879307,FLLU,HOL Double Play,,9359592,0,1,POINT (22.96285 40.60079),6.0,Thessaloniki,2359310886,9359592,2021-12-10,2021-12-08,24 Mbps,N/A,,ADSL
3,22.962849,40.600790,1026,21Β2560207,24576/1024,475.284308,367.079091,45827.0,0,103628488,2374422640,24/1,21Β2560207,FLLU,HOL Triple Play,,2663573,0,1,POINT (22.96285 40.60079),6.0,Thessaloniki,2370775223,2663573,2021-04-06,2021-03-31,24 Mbps,RETAIL,,ADSL
4,22.962849,40.600790,1026,21Β2583758,30720/3072,475.284308,367.079091,0.0,469,104042917,2374420673,30/3,21Β2583758,FLLU,HOL Double Play,VDSL OnNet,7210029,0,1,POINT (22.96285 40.60079),6.0,Thessaloniki,2374420673,7210029,2022-10-13,2022-10-13,30 Mbps,RETAIL,VDSL OnNet,30
5,22.962849,40.600790,1026,21Β2841585,24576/1024,475.291342,367.079091,45827.0,469,104726100,2350507520,24/1,21Β2841585,FLLU,HOL Double Play,,3311160,0,1,POINT (22.96285 40.60079),6.0,Thessaloniki,2350507520,3311160,2022-05-03,2022-05-03,24 Mbps,RETAIL,,ADSL
6,22.962849,40.600790,1026,21Β3922987,24576/1024,475.284308,367.079091,45827.0,469,105137284,2380533930,24/1,21Β3922987,FLLU,HOL Double Play,,0464998,469,1,POINT (22.96285 40.60079),6.0,Thessaloniki,2351276615,0464998,2022-01-22,2022-01-21,24 Mbps,RETAIL,,ADSL
7,23.678657,38.000120,463,21Β1545152,51200/5120,1124.222868,749.481912,0.0,126,102774048,2159125121,50/5,21Β1545152,FLLU,HOL Double Play,WIND VPU Light,507533,126,1,POINT (23.67866 38.00012),41.0,Attiki,2159125121,507533,2021-10-26,2021-10-24,50 Mbps,RETAIL,WIND VPU Light,50
8,20.487012,38.182229,1467,21Β2268988,51200/5120,559.434712,442.030755,38788.0,114,103260943,2646242046,50/5,21Β2268988,FLLU,HOL Double Play,VDSL OffNet,325024,114,1,POINT (20.48701 38.18223),26.0,Kefallinia,2646242046,325024,2022-09-02,2022-09-02,50 Mbps,RETAIL,VDSL OffNet,50
9,20.487012,38.182229,1467,21Β2010069,51200/5120,559.434712,442.030755,38788.0,114,101741833,2634928363,50/5,21Β2010069,FLLU,HOL Double Play,VDSL OffNet,325582,114,1,POINT (20.48701 38.18223),26.0,Kefallinia,2634928363,325582,2022-08-30,2022-08-30,50 Mbps,RETAIL,VDSL OffNet,50


# SOOOOOOOOOOOOOOOOOOOOOOOOS

In [106]:
# 2. Hubs_fixed_base & cable_providers

In [107]:
# # INNER JOIN !!!
# fixed_base_hubs.shape, cable_providers.shape
# check = fixed_base_hubs.merge(cable_providers, on=['CO_code','kvID'])
# check.shape

In [108]:
# LEFT JOIN !!!
fixed_base_hubs.shape, cable_providers.shape
fixed_base_hubs_providers = fixed_base_hubs.merge(cable_providers, on=['CO_code','kvID'], how='left')
fixed_base_hubs_providers.shape

((794307, 30), (23620, 15))

(794307, 43)

In [109]:
fixed_base_hubs_providers['kvID_Found'] = np.where(fixed_base_hubs_providers['kvID'] !='0', 1,0)

In [110]:
# 3

In [111]:
uni_fixed_base_hubs_providers = fixed_base_hubs_providers.drop_duplicates(subset=['G_LATITUDE','G_LONGITUDE','LDAP_ID','CLI'])
uni_fixed_base_hubs_providers.shape

# INNER
# +test
fixed_base_hubs_providers_network = uni_fixed_base_hubs_providers.merge(fixed_network_overview, on='LDAP_ID')

(794307, 44)

In [112]:
# Assure that we have value for CLI
fixed_base_hubs_providers_network = fixed_base_hubs_providers_network[fixed_base_hubs_providers_network.CLI != '0']
fixed_base_hubs_providers_network = fixed_base_hubs_providers_network[fixed_base_hubs_providers_network.CLI != '']

In [113]:
def speed_fix(x):
    if x['DSL'] == "N/A":
        if x['1M_Actual_Speed'] < 24000:
            return 'ADSL'
        elif (x['1M_Actual_Speed'] > 24000) & (x['1M_Actual_Speed'] <= 30000):
            return '30'
        elif (x['1M_Actual_Speed'] > 30000) & (x['1M_Actual_Speed'] <= 56000):
            return '50'        
        elif (x['1M_Actual_Speed'] > 56000) & (x['1M_Actual_Speed'] <= 110000):
            return '100'        
        else:
            return '200'
    else:
        return x['DSL']

In [114]:
fixed_base_hubs_providers_network['DSL'] = fixed_base_hubs_providers_network.apply(speed_fix, axis=1)

In [115]:
fixed_base_hubs_providers_network = fixed_base_hubs_providers_network[fixed_base_hubs_providers_network['CO_kv_Dist'] != '-']
fixed_base_hubs_providers_network['CO_kv_Dist'] = fixed_base_hubs_providers_network['CO_kv_Dist'].astype(float)

In [116]:
vis_df = fixed_base_hubs_providers_network

In [117]:
vis_df

,G_LONGITUDE,G_LATITUDE,CO_code,TOB,BANDWIDTH,G_WALKINGDST,G_RADIUSDST,VDSLESTIMATION,kvID,USER_NUM,PHONE,BANDWIDTH_1,ID_TOB,CIRCUIT_TYPE,APPLICATION_PRODUCT,VDSL_TECHNOLOGY,LDAP_ID,KVID_1,Neg_Latlong_Flag,geometry,index_right,ADMIN_NAME,CLI,LDAPID,SUBSCR_RENEWAL_DTTM,CONTRACT_START_DT,SPEED,VF_SEGMENT,VDSL_TYPE,DSL,City,County,Longtitude,Latitude,Provider,VDSL50,VDSL100,VDSL200,Connections,Length,CO_kv_Dist,Type,VDSL_SUM,kvID_Found,1M_MODE_SPEED,1M_Max_Actual_Speed,1M_Min_Actual_Speed,1M_Actual_Speed,1M_AttainableRate,1M_Attenuation,1M_ttl_disconnections,1M_ttl_AdminResets,1M_ttl_HostRequest,1M_ttl_LostCarrier,1M_ttl_LostService,1M_ttl_NasError,1M_ttl_NasRequest,1M_ttl_PortError,1M_ttl_SessionTimeOut,1M_ttl_UserError,1M_ttl_UserRequest,1M_ttl_INSESSIONTIME,1M_INSESSIONTIME,2M_MODE_SPEED,2M_Max_Actual_Speed,2M_Min_Actual_Speed,2M_Actual_Speed,2M_AttainableRate,2M_Attenuation,2M_ttl_disconnections,2M_ttl_AdminResets,2M_ttl_HostRequest,2M_ttl_LostCarrier,2M_ttl_LostService,2M_ttl_NasError,2M_ttl_NasRequest,2M_ttl_PortError,2M_ttl_SessionTimeOut,2M_ttl_UserError,2M_ttl_UserRequest,2M_ttl_INSESSIONTIME,2M_INSESSIONTIME,3M_MODE_SPEED,3M_Max_Actual_Speed,3M_Min_Actual_Speed,3M_Actual_Speed,3M_AttainableRate,3M_Attenuation,3M_ttl_disconnections,3M_ttl_AdminResets,3M_ttl_HostRequest,3M_ttl_LostCarrier,3M_ttl_LostService,3M_ttl_NasError,3M_ttl_NasRequest,3M_ttl_PortError,3M_ttl_SessionTimeOut,3M_ttl_UserError,3M_ttl_UserRequest,3M_ttl_INSESSIONTIME,3M_INSESSIONTIME,1M_Attenuation_10,2M_Attenuation_10,3M_Attenuation_10,1M_MODE_SPEED_1000,2M_MODE_SPEED_1000,3M_MODE_SPEED_1000,1M_Actual_Speed_1000,2M_Actual_Speed_1000,3M_Actual_Speed_1000,1M_Max_Actual_Speed_1000,2M_Max_Actual_Speed_1000,3M_Max_Actual_Speed_1000,1M_Min_Actual_Speed_1000,2M_Min_Actual_Speed_1000,3M_Min_Actual_Speed_1000,ceil_1M_Attenuation_10,ceil_2M_Attenuation_10,ceil_3M_Attenuation_10,AVG_2M_MODE_SPEED,AVG_3M_MODE_SPEED,AVG_2M_Actual_Speed,AVG_3M_Actual_Speed,AVG_2M_Attenuation,AVG_3M_Attenuation,AVG_2M_ttl_disconnections,AVG_3M_ttl_disconnections
0,22.962849,40.600790,1026,21Β1884869,30720/3072,475.284308,367.079091,0.0,469,101696921,2359243648,30/3,21Β1884869,FLLU,HOL Double Play,VDSL OnNet,643234,0,1,POINT (22.96285 40.60079),6.0,Thessaloniki,2359243648,643234,2022-12-13,2022-12-13,30 Mbps,RETAIL,VDSL OnNet,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,31212.0,31215.0,30656.0,31000.064516,35741.000000,32.000000,41,0,0,40,0,0,0,0,0,0,1,1343709,43345.451613,30720.0,31022.0,28652.0,30555.250000,35395.464286,32.000000,44,0,0,44,0,0,0,0,0,0,0,1112473,39731.178571,28652.0,28655.0,23532.0,27496.354839,35106.774194,31.809677,36,0,0,36,0,0,0,0,0,0,0,1977162,63779.419355,3.200000,3.200000,3.180968,31.212,30.720,28.652,31.000065,30.555250,27.496355,31.215,31.022,28.655,30.656,28.652,23.532,4,4,4,30.966,30.194667,30.777657,29.683890,4.0,4.0,42.5,40.333333
1,22.962849,40.600790,1026,21Β2576887,24576/1024,475.291342,367.079091,24986.0,469,104779182,2343990804,24/1,21Β2576887,FLLU,HOL Double Play,,1448348,0,1,POINT (22.96285 40.60079),6.0,Thessaloniki,2343990804,1448348,2021-07-14,2021-07-14,24 Mbps,N/A,,ADSL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,15052.0,15052.0,15052.0,15052.000000,16421.476190,14.600000,3,0,0,2,0,0,0,0,0,0,1,2678311,86397.129032,14572.0,16382.0,13873.0,14956.590909,16534.636364,14.700000,9,0,0,4,0,0,0,0,0,0,5,2505267,86388.517241,16382.0,16382.0,16382.0,16382.000000,18337.142857,14.625000,2,0,0,2,0,0,0,0,0,0,0,2670708,86151.870968,1.460000,1.470000,1.462500,15.052,14.572,16.382,15.052000,14.956591,16.382000,15.052,16.382,16.382,15.052,13.873,16.382,2,2,2,14.812,15.335333,15.004295,15.463530,2.0,2.0,6.0,4.666667
2,22.962849,40.600790,1026,21Β3879307,24576/1024,475.291342,367.079091,28310.0,469,104779327,2141377472,24/1,21Β3879307,FLLU,HOL Double Play,,9359592,0,1,POINT (22.96285 40.60079),6.0,Thessaloniki,2359310886,9359592,2021-12-10,2021-12-08,24 Mbps,N/A,,ADSL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,12288.0,14135.0,12243

# ANALYSIS PER KVID

In [118]:
# Calculate some aggregations for distance, speed, attenuation, disconnections for 50 & 100
kvid_agg1 = vis_df[vis_df.G_LATITUDE != 0.0].groupby(['CO_code','kvID','DSL'])[['1M_Actual_Speed', '1M_ttl_disconnections']].mean().reset_index()

kvid_agg2 = vis_df[vis_df.G_LATITUDE != 0.0].groupby(['CO_code','kvID','DSL'])[['1M_Actual_Speed', '1M_ttl_disconnections']].std().reset_index()

kvid_agg3 = vis_df[vis_df.G_LATITUDE != 0.0].groupby(['CO_code','kvID','DSL'])['1M_Actual_Speed'].count().rename("num_of_connections").reset_index()

In [119]:
kvid_agg3

,CO_code,kvID,DSL,num_of_connections
0,0,0,100,3
1,0,0,50,29
2,0,0,ADSL,4
3,0,101,50,4
4,0,101,ADSL,1
...,...,...,...,...
76348,995,405,ADSL,2
76349,995,407,50,4
76350,996,113,50,1
76351,996,403,100,1


In [120]:
# create a dictionary
# key = old name
# value = new name
dict1 = {'1M_Actual_Speed': 'mean_Actual_Speed',
        '1M_ttl_disconnections': 'mean_disconnections'}

dict2 = {'1M_Actual_Speed': 'std_Actual_Speed',
        '1M_ttl_disconnections': 'std_disconnections'}
 
# call rename () method
kvid_agg1.rename(columns=dict1,
          inplace=True)

kvid_agg2.rename(columns=dict2,
          inplace=True)

In [121]:
kvid_aggs = kvid_agg1.merge(kvid_agg2, on=['CO_code','kvID','DSL']).merge(kvid_agg3, on=['CO_code','kvID','DSL'])

In [122]:
kvid_aggs = kvid_aggs.fillna(0.0)

In [123]:
kvid_aggs_uni = vis_df[['CO_code','kvID','DSL', 'Type','Provider','Connections']].drop_duplicates()
kvid_aggs_uni.shape

(78006, 6)

In [124]:
kvid_aggs = kvid_aggs.merge(kvid_aggs_uni[['CO_code','kvID','DSL','Type','Provider','Connections']], on=['CO_code','kvID','DSL'])
kvid_aggs.shape

(76353, 11)

In [125]:
kvid_aggs['mean&std_Actual_Speed'] = kvid_aggs['mean_Actual_Speed'] - (kvid_aggs['std_Actual_Speed']*0.35)  # exw kanei fill nans me 0 

In [126]:
kvid_aggs = kvid_aggs.add_prefix('kvID_')

In [127]:
kvid_aggs.rename(columns={'kvID_CO_code':'CO_code','kvID_kvID':'kvID','kvID_DSL':'DSL'}, inplace=True)

In [128]:
kvid_aggs

,CO_code,kvID,DSL,kvID_mean_Actual_Speed,kvID_mean_disconnections,kvID_std_Actual_Speed,kvID_std_disconnections,kvID_num_of_connections,kvID_Type,kvID_Provider,kvID_Connections,kvID_mean&std_Actual_Speed
0,0,0,100,108040.572797,1.333333,1715.738500,1.154701,3,NaN,NaN,NaN,107440.064322
1,0,0,50,46794.973602,6.103448,12396.433372,6.212573,29,NaN,NaN,NaN,42456.221922
2,0,0,ADSL,8665.717358,17.500000,4229.810624,8.103497,4,NaN,NaN,NaN,7185.283639
3,0,101,50,26841.016129,13.500000,19797.944805,12.476645,4,NaN,NaN,NaN,19911.735447
4,0,101,ADSL,14567.000000,0.000000,0.000000,0.000000,1,NaN,NaN,NaN,14567.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
76348,995,405,ADSL,17912.208565,2.000000,993.121642,1.414214,2,ΑΣΤΙΚΟ,ΟΤΕ,91.0,17564.615990
76349,995,407,50,54345.693548,32.250000,755.644985,57.835255,4,ΑΣΤΙΚΟ,ΟΤΕ,103.0,54081.217804
76350,996,113,50,12614.387097,3.000000,0.000000,0.000000,1,ΗΜΙΑΣΤΙΚΟ,ΟΤΕ,223.0,12614.387097
76351,996,403,100,107575.935484,7.000000,0.000000,0.000000,1,ΗΜΙΑΣΤΙΚΟ,ΟΤΕ,131.0,107575.935484


In [129]:
kvid_aggs.shape
kvid_coords.shape

(76353, 12)

(23620, 4)

In [130]:
kvid_aggs["CO_code"].nunique()

841

In [131]:
kvid_coords["CO_code"].nunique()

1248

In [132]:
#kvid_aggs['CO_code'] = kvid_aggs['CO_code'].apply(lambda x:str(int(float(x))))

In [133]:
kvid_aggs_final = kvid_aggs.merge(kvid_coords, on=["CO_code", "kvID"], how='left')

In [134]:
kvid_aggs_final.shape

(76353, 14)

In [135]:
kvid_aggs_final["CO_code"].nunique()
kvid_aggs_final["kvID"].nunique()

841

1085

In [136]:
kvid_aggs_final

,CO_code,kvID,DSL,kvID_mean_Actual_Speed,kvID_mean_disconnections,kvID_std_Actual_Speed,kvID_std_disconnections,kvID_num_of_connections,kvID_Type,kvID_Provider,kvID_Connections,kvID_mean&std_Actual_Speed,Longtitude,Latitude
0,0,0,100,108040.572797,1.333333,1715.738500,1.154701,3,NaN,NaN,NaN,107440.064322,NaN,NaN
1,0,0,50,46794.973602,6.103448,12396.433372,6.212573,29,NaN,NaN,NaN,42456.221922,NaN,NaN
2,0,0,ADSL,8665.717358,17.500000,4229.810624,8.103497,4,NaN,NaN,NaN,7185.283639,NaN,NaN
3,0,101,50,26841.016129,13.500000,19797.944805,12.476645,4,NaN,NaN,NaN,19911.735447,NaN,NaN
4,0,101,ADSL,14567.000000,0.000000,0.000000,0.000000,1,NaN,NaN,NaN,14567.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76348,995,405,ADSL,17912.208565,2.000000,993.121642,1.414214,2,ΑΣΤΙΚΟ,ΟΤΕ,91.0,17564.615990,23.705083,40.717083
76349,995,407,50,54345.693548,32.250000,755.644985,57.835255,4,ΑΣΤΙΚΟ,ΟΤΕ,103.0,54081.217804,23.697417,40.710500
76350,996,113,50,12614.387097,3.000000,0.000000,0.000000,1,ΗΜΙΑΣΤΙΚΟ,ΟΤΕ,223.0,12614.387097,23.702444,40.664417
76351,996,403,100,107575.935484,7.000000,0.000000,0.000000,1,ΗΜΙΑΣΤΙΚΟ,ΟΤΕ,131.0,107575.935484,23.691056,40.666583


In [137]:
kvid_aggs_fin =kvid_aggs_final[["CO_code", "kvID", "DSL", "kvID_mean_Actual_Speed", "kvID_mean_disconnections", "kvID_num_of_connections", "Longtitude", "Latitude"]]

In [138]:
kvid_aggs_fin

,CO_code,kvID,DSL,kvID_mean_Actual_Speed,kvID_mean_disconnections,kvID_num_of_connections,Longtitude,Latitude
0,0,0,100,108040.572797,1.333333,3,NaN,NaN
1,0,0,50,46794.973602,6.103448,29,NaN,NaN
2,0,0,ADSL,8665.717358,17.500000,4,NaN,NaN
3,0,101,50,26841.016129,13.500000,4,NaN,NaN
4,0,101,ADSL,14567.000000,0.000000,1,NaN,NaN
...,...,...,...,...,...,...,...,...
76348,995,405,ADSL,17912.208565,2.000000,2,23.705083,40.717083
76349,995,407,50,54345.693548,32.250000,4,23.697417,40.710500
76350,996,113,50,12614.387097,3.000000,1,23.702444,40.664417
76351,996,403,100,107575.935484,7.000000,1,23.691056,40.666583


In [139]:
kvid_aggs_fin.isnull().values.any()

True

In [140]:
kvid_aggs_fin.to_csv("gs://vf-gr-ca-nonlive-devds/irida/fixed_visualization/co_code_kv_aggs.csv", index=False)